1. 将posts和replys数据集导入R中，命名为r.posts和r.replys，保留所有字段名，所有字段空⽩转化
为NA。

In [9]:
library(data.table)
library(stringr)
r.posts <- fread("posts.csv", na.strings = c("", " "),fill = TRUE)
r.replys <- fread("replys.csv", na.strings = c("", " "),fill = TRUE)

Warning message in fread("posts.csv", na.strings = c("", " "), fill = TRUE):
"na.strings[2]==" " consists only of whitespace, ignoring. strip.white==TRUE (default) and "" is present in na.strings, so any number of spaces in string columns will already be read as <NA>."
Warning message in fread("replys.csv", na.strings = c("", " "), fill = TRUE):
"na.strings[2]==" " consists only of whitespace, ignoring. strip.white==TRUE (default) and "" is present in na.strings, so any number of spaces in string columns will already be read as <NA>."


2. 对r.posts进⾏操作:
删除所有postnums字段都为0的⾏，查看post_id的数量。
统计字段guba_name为**“螺纹钢吧”**帖⼦的数量，与postnums进⾏对比，哪⼀个更⼤。
按create_time进⾏升序排列，找出每个吧中最早创建的帖⼦，将所有这些帖⼦的信息导出⽣成⼀个表,
命名为earl.posts。

In [10]:
r.posts <- r.posts[postnums != 0]
nrow(r.posts)
luowennum.count <- nrow(r.posts[guba_name == "螺纹钢吧"])
postnums.sum <- sum(r.posts$postnums)  # 统计所有postnums的总和
cat("螺纹钢吧帖子数量：", luowennum.count, "；postnums总和：", postnums.sum, "\n")
cat("两者对比：", ifelse(luowennum.count > postnums.sum, "螺纹钢吧帖子数量更大", "postnums总和更大"), "\n")
r.posts[, create_time := as.POSIXct(create_time)]
earl.posts <- r.posts[order(create_time), .SD[1], by = guba_name]
fwrite(earl.posts, "earl.posts.csv")

[1] 253092

螺纹钢吧帖子数量： 23209 ；postnums总和： 21590597297 
两者对比： postnums总和更大 


3. 对r.replys进⾏操作：
按照post_id和reply_time进⾏分组升序排列，导出最晚的⼀条回复，⽣成⼀个表，名为
late.replys。
作业5.md 2025-11-16
2 / 2
删除所有reply_content中带有’<>’的部分，例如
<imgsrc=""http://gbres.dfcfw.com/face/emot/emot14.png""title=""⽜"">，将此类内容删除。（提⽰：
使⽤stringr这个package，具体stringr的⽤法请参考https://stringr.tidyverse.org/或向AI提问）

In [11]:
r.replys[, reply_time := as.POSIXct(reply_time)]
late.replys <- r.replys[order(post_id, reply_time), .SD[.N], by = post_id]
fwrite(late.replys, "late.replys.csv")
r.replys[, reply_content := str_remove_all(reply_content, "<[^>]+>")]

4. 上述操作完成之后，对已经处理好的r.posts和r.replys，以post_id这个字段进⾏合并，⽣成⼀个
表，命名为int.data。要求每⼀条reply都对应⼀条帖⼦的信息，也就是将帖⼦中的reply全部展开，
删除在两表中⽆法匹配的条⽬。

In [12]:
int.data <- merge(r.posts, r.replys, by = "post_id")
fwrite(int.data, "int.data.csv")

5. 提取int.data中create_time的年⽉⽇部分，⽣成⼀个新变量create_date，以"%Y-%m-%d"的Date格
式储存；将create_time改为POSXct的标准时间格式："%Y-%m-%d %H:%M:%OS"。（提⽰：使⽤
stringr这个package，具体stringr的⽤法请参考https://stringr.tidyverse.org/或向AI提问）
提取int.data中create_date的最⼤值和最⼩值，提取所有guba_name，对每⼀个guba_name都⽣成⼀
个以⽇为单位的时间序列向量，区间为create_date的最⼤值和最⼩值。并最后输出⼀个表，命名为tim。
例如有三个guba_name：“⽟米吧”，“螺纹钢吧”，“财经评论吧”，create_date最⼩值和最⼤值分别
为“2015-5-15”和“2015-5-17”，则⽣成了9⾏观察值，每个⽇期都分别对应了每个吧。
将tim与int.data以guba_name和create_date进⾏合并，⽣成⼀个新表，命名为int.data.ts，将所
有为NA的readnum和replynum观测填充为0。

In [13]:
int.data[,create_date := as.Date(create_time,format = "%Y-%m-%d")]
int.data[, create_time := as.POSIXct(create_time,format="%Y-%m-%d %H:%M:%OS")]
min.date <- min(int.data$create_date)
max.date <- max(int.data$create_date)
guba.names <- unique(int.data$guba_name)
tim <- data.table(guba_name = rep(guba.names, each = as.integer(max.date - min.date + 1)),
create_date = rep(seq(min.date, max.date, by = "day"), length(guba.names)))
int.data.ts <- merge(tim, int.data, by = c("guba_name", "create_date"), all.x = TRUE)
int.data.ts[, `:=`(readnum = fifelse(is.na(readnum), 0, readnum), replynum = fifelse(is.na(replynum), 0, replynum))]
fwrite(int.data.ts, "int.data.ts.csv")

6. 统计分析
在r.posts中，提取出replynum最多的⼗条帖⼦和⼗个吧，⽣成两张表为mx.post和mx.ba，表⾥变量
分别包括帖⼦id、吧名、帖⼦阅读数和帖⼦回复数，吧名、吧总体阅读数和吧总体回复数。
在r.posts中，调⽤任何相关的描述性统计包，对帖⼦和吧做描述性统计，要求⾄少导出replynum和
readnum的最⼤值、最⼩值、均值、中位数和⽅差，并导成⼀张表名为stst.int。

In [16]:
mx.post <- r.posts[order(-replynum)][1:10, .(post_id, guba_name, readnum, replynum)]
mx.ba <- r.posts[, .(total_readnum = sum(readnum), total_replynum = sum(replynum)), by = guba_name][order(-total_replynum)][1:10]
fwrite(mx.post, "mx.post.csv")
fwrite(mx.ba, "mx.ba.csv")
stst.int.post <- r.posts[, .(replynum_max = max(replynum),
replynum_min = min(replynum), 
replynum_mean = mean(replynum),
replynum_median = median(replynum), 
replynum_var = var(replynum),
readnum_max = max(readnum),
readnum_min = min(readnum), 
readnum_mean = mean(readnum), 
readnum_median = median(readnum), 
readnum_var = var(readnum))]
stst.int.ba <- r.posts[, 
.(replynum_max = max(replynum), 
replynum_min = min(replynum), 
replynum_mean = mean(replynum), 
replynum_median = median(replynum), 
replynum_var = var(replynum),
readnum_max = max(readnum), 
readnum_min = min(readnum), 
readnum_mean = mean(readnum), 
readnum_median = median(readnum), 
readnum_var = var(readnum)), by = guba_name]
fwrite(rbindlist(list(post = stst.int.post, ba = stst.int.ba), idcol = "level", fill=TRUE), "stst.int.csv")

